### 1. Import liblaty

In [1]:
import pandas as pd
from googleapiclient.discovery import build

### 2. make fungsion for scrapping

In [6]:
def video_comments(video_id, max_results = 100):
    replies = []

    youtube = build("youtube", "v3", developerKey = api_key) 

    video_response = youtube.commentThreads().list(
        part = "snippet, replies",
        videoId = video_id, 
        maxResults = min(max_results, 100)
    ).execute()

    comment_count = 0

    while video_response and comment_count < max_results:
        for item in video_response.get("items", []): 
            
            published = item["snippet"]["topLevelComment"]["snippet"]["publishedAt"]
            user = item["snippet"]["topLevelComment"]["snippet"]["authorDisplayName"]
            comment = item["snippet"]["topLevelComment"]["snippet"]["textDisplay"]
            likeCount = item["snippet"]["topLevelComment"]["snippet"]["likeCount"]

            replies.append([published, user, comment, likeCount])
            comment_count += 1

            replycount = item["snippet"]["totalReplyCount"]

            if replycount > 0:
                if "replies" in item:
                    for reply in item["replies"]["comments"]:
                        published = reply["snippet"]["publishedAt"]
                        user = reply["snippet"]["authorDisplayName"]
                        repl = reply["snippet"]["textDisplay"]
                        likeCount = reply["snippet"]["likeCount"]

                        replies.append([published, user, repl, likeCount])
                        comment_count +=1
                        if comment_count >= max_results:
                            break
            if comment_count >= max_results:
                break
        
        if "nextPageToken" in video_response and comment_count < max_results:
            video_response = youtube.commentThreads().list(
                part = "snippet,replies",
                pageToken = video_response["nextPageToken"],
                videoId = video_id, 
                maxResults = min(max_results - comment_count, 100)
            ).execute()
        else:
            break
            
    return replies

### 3. run proses scrapping

In [9]:
api_key = "api_key"

video_id = "3UTMXKg47BM"

jumlah_data = 422

comments = video_comments(video_id, jumlah_data)

In [14]:
print(f"jumlah data yang berhasil diambil adalah : {len(comments)}")

jumlah data yang berhasil diambil adalah : 407


### 4. to dataframe

In [13]:
df = pd.DataFrame(comments, columns=["Date", "Username", "Comment", "Like"])
df

,Date,Username,Comment,Like
0,2025-11-24T01:13:15Z,@hendratmobudihardjo4405,"Gak papa pak, biar saingan kita sebagai progra...",0
1,2025-11-23T10:30:24Z,@TigaherangChannel,"Dari sekian banyak AI, sejauh ini gw baru pake...",0
2,2025-11-23T02:05:51Z,@noertri618,Gara-gara AI harga komponen-komponen pc naik g...,0
3,2025-11-21T15:27:06Z,@kuliahkentang2146,"Kalo pas skripsi belom bisa coding, mau beli p...",0
4,2025-11-18T09:04:50Z,@nengmoemoen,"Saya punya pengalaman, d kantor sayaadassorang...",0
...,...,...,...,...
402,2025-10-13T11:32:02Z,@muhammad.azqilana,Ini yg aku tunggu terimakasih pak pengajaran d...,1
403,2025-10-13T11:31:38Z,@parthawijaya482,"Ijin menambahkan Pak Sandhika, vibe coder itu ...",0
404,2025-10-13T12:56:40Z,@sandhikagalihWPU,"nice share. yang jago makin jago, yang ngecit ...",0
405,2025-10-13T11:30:57Z,@rofiaziztaufikrahmat,First,0


### 5. save to csv and excel

In [15]:
df.to_excel("data-analisis-sentimen-terhadap-fenomena-vibecoding.xlsx", index = False)
df.to_csv("data-analisis-sentimen-terhadap-fenomena-vibecoding.csv", index = False)